# 🚀 ADK Adventure: External User-Level Database 🧑‍💼

Welcome, Agent Architect! In this notebook, we're building the most robust memory pattern: an **External User-Level Database**.

Here's the architecture:
1.  **`DatabaseSessionService` (for Conversation):** We will still use the ADK's built-in service to manage *conversational history* (what was said in this specific chat). This is stored in `colab_persistent_sessions.db`.
2.  **External SQLite DB (for User Facts):** We will create a *separate* database (`user_preferences.db`) to store permanent facts about the user (like "I am vegetarian").

Our agent's tools (`save_user_preferences`, `recall_user_preferences`) will now connect *directly* to this external DB, using the `user_id` as a key. This means **any session** from the **same user** can instantly access their global preferences.

-------------
### ⚠️ Important Prerequisite: Setup Your Environment! ⚠️
-----------------------------------------------------------------------------
 Before you run this file, please make sure you have completed the initial
setup steps from the Google Codelabs instructions.

This includes:

1. Setting up a Google Cloud project with billing.
2. Getting your free Gemini API Key.

👉 Follow the official guide here:
   https://codelabs.developers.google.com/onramp/instructions#0
 -----------------------------------------------------------------------------

## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries (including `sqlalchemy` for the session service) and securely configures your Google API key.

In [ ]:
# We need sqlalchemy to use the DatabaseSessionService with SQLite
!pip install google-adk google-generativeai sqlalchemy google-cloud-aiplatform -q

# --- Import all necessary libraries ---
import os
import asyncio
import logging
import sqlite3
import json
from pathlib import Path
from getpass import getpass
from typing import AsyncGenerator, Dict, Any

# Google & ADK Imports
import google.generativeai as genai
from google.genai.types import Content, Part
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.events import Event
from google.adk.agents.invocation_context import InvocationContext
from google.adk.tools import ToolContext, google_search
from google.adk.tools.agent_tool import AgentTool
from google.adk.runners import Runner
from google.adk.sessions import DatabaseSessionService
from typing_extensions import override

# IPython imports for rich display in Colab
from IPython.display import display, Markdown

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [ ]:
auth.authenticate_user()

In [ ]:
# fmt: on
PROJECT_ID = "placeholder" # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
LOCATION = "us-central1"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"

---
## 1. Configure the (Conversational) Session Service 💬

This first database will be used by the ADK `Runner` to store the *history of the conversation* (the back-and-forth messages). This is **still required** for the agent to have conversational context.

In [ ]:
# --- Configuration ---
# This DB stores the CONVERSATION HISTORY
SESSIONS_DIR = Path(os.path.expanduser("~")) / ".adk" / "sessions"
os.makedirs(SESSIONS_DIR, exist_ok=True)
SESSION_DB_FILE = SESSIONS_DIR / "colab_persistent_sessions.db"
SESSION_URL = f"sqlite:///{SESSION_DB_FILE}"

# We'll define a unique user and session ID
MY_USER_ID = "colab_user_001"
MY_SESSION_ID = f"{MY_USER_ID}_session_a" # We can change this to test isolation!

# --- Initialize the Service ---
session_service = DatabaseSessionService(db_url=SESSION_URL)

print(f"🤖 Initializing Personalized Trip Planner...")
print(f"💬 Conversation DB is configured at: {SESSION_DB_FILE}")
print("✅ DatabaseSessionService is ready.")

🤖 Initializing Personalized Trip Planner...
💬 Conversation DB is configured at: /root/.adk/sessions/colab_persistent_sessions.db
✅ DatabaseSessionService is ready.


---
## 2. Setup the External User-Level Database 🧑‍💼

This second database is for our *tools*. This is where we will store **permanent, global facts** about the user, like dietary preferences or a home city. Notice it's a totally separate file.

In [ ]:
# This DB stores the USER'S PROFILE
USER_DB_FILE = "user_preferences.db"

# Connect and create the table if it doesn't exist
try:
    with sqlite3.connect(USER_DB_FILE) as conn:
        cursor = conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS user_preferences (
            id INTEGER PRIMARY KEY,
            user_id TEXT NOT NULL,
            pref_key TEXT NOT NULL,
            pref_value TEXT NOT NULL,
            UNIQUE(user_id, pref_key) -- Ensures one value per key per user
        );
        """)
        conn.commit()
        print(f"🧑‍💼 User preference database '{USER_DB_FILE}' is ready.")
except Exception as e:
    print(f"Error setting up user DB: {e}")

🧑‍💼 User preference database 'user_preferences.db' is ready.


---
## 3. Define the Agents 🤖

Now we'll define our agents and memory tools.

Pay close attention to `save_user_preferences` and `recall_user_preferences`. They **do not** use `tool_context.state`. Instead, they get the `user_id` from the context and then use `sqlite3` to connect to our `user_preferences.db` file directly.

In [ ]:
# Make sure this matches the cell above
USER_DB_FILE = "user_preferences.db"

# --- 1. Define the USER-LEVEL Memory Tools ---

def save_user_preferences(tool_context: ToolContext, new_preferences: Dict[str, Any]) -> str:
    """
    Saves or updates user preferences in the EXTERNAL user_preferences.db.
    """
    # Get the user_id for this session
    user_id = tool_context.session.user_id
    if not user_id:
        return "Error: Could not identify user."

    print(f"\n[DEBUG] Tool 'save_user_preferences' connecting to {USER_DB_FILE} for user {user_id}")
    try:
        with sqlite3.connect(USER_DB_FILE) as conn:
            cursor = conn.cursor()
            for key, value in new_preferences.items():
                # Store complex values (like lists) as JSON strings
                value_str = json.dumps(value)
                cursor.execute("""
                INSERT INTO user_preferences (user_id, pref_key, pref_value)
                VALUES (?, ?, ?)
                ON CONFLICT(user_id, pref_key) DO UPDATE SET pref_value = excluded.pref_value;
                """, (user_id, key, value_str))
            conn.commit()
        return f"Preferences updated successfully in user DB: {new_preferences}"
    except Exception as e:
        return f"Error saving preferences to DB: {e}"

def recall_user_preferences(tool_context: ToolContext) -> Dict[str, Any]:
    """Recalls all saved preferences for the current user from the EXTERNAL user_preferences.db."""
    user_id = tool_context.session.user_id
    if not user_id:
        return {"message": "Error: Could not identify user."}

    print(f"\n[DEBUG] Tool 'recall_user_preferences' connecting to {USER_DB_FILE} for user {user_id}")
    preferences = {}
    try:
        with sqlite3.connect(USER_DB_FILE) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT pref_key, pref_value FROM user_preferences WHERE user_id = ?", (user_id,))
            rows = cursor.fetchall()

            if not rows:
                return {"message": "No preferences found in user DB for this user."}

            for key, value_str in rows:
                # Load the JSON string back into a Python object
                preferences[key] = json.loads(value_str)
        return preferences
    except Exception as e:
        return {"message": f"Error recalling preferences from DB: {e}"}


# --- 2. Define the Specialist "Tool" Agent ---
planner_tool_agent = LlmAgent(
    name="PlannerToolAgent",
    model="gemini-2.5-flash",
    description="A specialist that finds activities and restaurants based on a user's request and preferences.",
    instruction="""
    You are a planning assistant. Based on the user's request and their provided preferences (e.g., 'vegetarian'), find one activity and one restaurant in Sunnyvale.
    If the user has a dietary preference, make sure the restaurant matches it.
    Output the plan as a simple JSON object.
    Example: {"activity": "The Tech Interactive", "restaurant": "Il Postale"}
    """,
    tools=[google_search]
)

# --- 3. Define the Main Coordinator Agent (Unchanged) ---
root_agent = LlmAgent(
    name="MemoryCoordinatorAgent",
    model="gemini-2.5-flash",
    instruction="""
    You are a highly intelligent, personalized trip planner with a persistent memory.
    1. RECALL FIRST: At the absolute beginning of the conversation, your first action MUST be to call the `recall_user_preferences` tool.
    2. PERSONALIZE & PLAN: Use the recalled preferences to enrich the user's current request. Call the `PlannerToolAgent` with the combined request.
    3. PRESENT & LEARN: Present the plan. Then, ask for feedback and if there are any new preferences you should save.
    4. SAVE LAST: If the user provides a new preference, your final action MUST be to call the `save_user_preferences` tool.
    """,
    tools=[
        recall_user_preferences,
        save_user_preferences,
        AgentTool(agent=planner_tool_agent)
    ]
)

print("🤖 Memory Coordinator Agent (with EXTERNAL User-Level DB) is ready.")

🤖 Memory Coordinator Agent (with EXTERNAL User-Level DB) is ready.


---
## 4. Create a Colab-Friendly Runner 🏃

This helper function is the same as before. It will initialize the `Runner` with our `DatabaseSessionService` (for conversation history) and manage running the queries.

In [ ]:
# --- Initialize the Runner with our Persistent Service ---
runner = Runner(
    agent=root_agent,
    session_service=session_service,
    app_name=root_agent.name
)

# Helper function to get or create our one persistent session
async def get_or_create_session(user_id, session_id):
    session = await session_service.get_session(
        app_name=root_agent.name, user_id=user_id, session_id=session_id
    )
    if session is None:
        print(f"\n[SYSTEM] No existing conversation. Creating new one: {session_id}")
        session = await session_service.create_session(
            app_name=root_agent.name, user_id=user_id, session_id=session_id
        )
    else:
        print(f"\n[SYSTEM] ✅ Found existing conversation: {session.id}")
    return session

# This function adapts your main.py's event loop for Colab
async def run_persistent_query_async(query: str, user_id: str, session_id: str):
    print(f"\n--- 🗣️ You: {query} ---")
    print("🤖 Agent: ", end="", flush=True)

    # Get our persistent session
    session = await get_or_create_session(user_id, session_id)

    final_response_text = ""

    # Use the robust event loop from your main.py
    try:
        async for event in runner.run_async(
            user_id=session.user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not event.content:
                continue

            for part in event.content.parts:
                # Case 1: Text chunk from the model
                if hasattr(part, "text") and part.text:
                    print(part.text, end="", flush=True)
                    if event.is_final_response():
                        final_response_text = part.text

                # Case 2: Tool result (for debugging)
                elif event.author == "user" and hasattr(part, "function_response"):
                    tool_name = part.function_response.name
                    tool_result = part.function_response.response

                    # Print debug info on a new line (our custom tool logs will also appear here)
                    print(f"\n[DEBUG] Tool '{tool_name}' returned: {tool_result}")

                    # Re-print the "Agent:" prompt
                    print("\nAgent: ", end="", flush=True)

        # Add a final newline for clean formatting
        print("\n" + "-"*50)

    except Exception as e:
        print(f"\n\nAn error occurred: {e}")
        print("\n" + "-"*50)

    return final_response_text

print("✅ Colab runner function is defined.")

✅ Colab runner function is defined.


---
## 5. Run the Demonstration (Part 1) 🧪

Let's have a conversation. We will use `MY_USER_ID` and a session ID called `session_a`.

1.  **Turn 1:** We'll ask for help. The agent should call `recall_user_preferences` and find nothing in `user_preferences.db`.
2.  **Turn 2:** We'll give it a preference. The agent should call `save_user_preferences`, which writes to `user_preferences.db`.
3.  **Turn 3:** We'll ask for a plan. The agent should call `recall_user_preferences` and find our preference!

In [ ]:
# Turn 1: Initial query
# The agent should find NO preferences in the external DB.
query1 = "Can you help me plan a trip?"
await run_persistent_query_async(query1, MY_USER_ID, MY_SESSION_ID)


--- 🗣️ You: Can you help me plan a trip? ---
🤖 Agent: 
[SYSTEM] No existing conversation. Creating new one: colab_user_001_session_a



[DEBUG] Tool 'recall_user_preferences' connecting to user_preferences.db for user colab_user_001


Yes, I can help you plan a trip! To start, I need a little more information. Please tell me:

1.  **What is your destination?** (e.g., city, state, country)
2.  **What are your dates of travel?** (optional, but helpful for checking availability)
3.  **Are there any specific activities or types of food you're interested in?** (e.g., museums, outdoor activities, vegetarian restaurants, fine dining)
4.  **Do you have any dietary preferences or restrictions?** (e.g., vegetarian, vegan, gluten-free)
5.  **Is there anything else I should know to help you plan your trip?**

Once I have this information, I can start putting together a personalized plan for you! Do you have any feedback on these questions or any new preferences you'd like me to save for future trips?
--------------------------------------------------


"Yes, I can help you plan a trip! To start, I need a little more information. Please tell me:\n\n1.  **What is your destination?** (e.g., city, state, country)\n2.  **What are your dates of travel?** (optional, but helpful for checking availability)\n3.  **Are there any specific activities or types of food you're interested in?** (e.g., museums, outdoor activities, vegetarian restaurants, fine dining)\n4.  **Do you have any dietary preferences or restrictions?** (e.g., vegetarian, vegan, gluten-free)\n5.  **Is there anything else I should know to help you plan your trip?**\n\nOnce I have this information, I can start putting together a personalized plan for you! Do you have any feedback on these questions or any new preferences you'd like me to save for future trips?"

In [ ]:
# Turn 2: Save a preference
# The agent should call save_user_preferences, which writes to user_preferences.db
query2 = "Great. Please save this preference: I am a vegetarian."
await run_persistent_query_async(query2, MY_USER_ID, MY_SESSION_ID)


--- 🗣️ You: Great. Please save this preference: I am a vegetarian. ---
🤖 Agent: 
[SYSTEM] ✅ Found existing conversation: colab_user_001_session_a



[DEBUG] Tool 'save_user_preferences' connecting to user_preferences.db for user colab_user_001
Great! I've saved your preference that you are a vegetarian. I will keep this in mind for all future trip planning.

Now, going back to planning your current trip, could you please provide the following details?

1.  **What is your destination?** (e.g., city, state, country)
2.  **What are your dates of travel?** (optional, but helpful for checking availability)
3.  **Are there any specific activities or types of food you're interested in?** (e.g., museums, outdoor activities, fine dining - keeping in mind your vegetarian preference!)
4.  **Is there anything else I should know to help you plan your trip?**
--------------------------------------------------


"Great! I've saved your preference that you are a vegetarian. I will keep this in mind for all future trip planning.\n\nNow, going back to planning your current trip, could you please provide the following details?\n\n1.  **What is your destination?** (e.g., city, state, country)\n2.  **What are your dates of travel?** (optional, but helpful for checking availability)\n3.  **Are there any specific activities or types of food you're interested in?** (e.g., museums, outdoor activities, fine dining - keeping in mind your vegetarian preference!)\n4.  **Is there anything else I should know to help you plan your trip?**"

In [ ]:
# Turn 3: Ask for a plan
# The agent should RECALL our preference from the external DB and use it.
query3 = "Now, can you plan a trip to Sunnyvale?"
await run_persistent_query_async(query3, MY_USER_ID, MY_SESSION_ID)


--- 🗣️ You: Now, can you plan a trip to Sunnyvale? ---
🤖 Agent: 
[SYSTEM] ✅ Found existing conversation: colab_user_001_session_a


Great! For your trip to Sunnyvale, I recommend the following:

*   **Activity:** Computer History Museum
*   **Restaurant:** Merit Vegan Restaurant (keeping in mind your vegetarian preference!)

Would you like to explore more options for activities or dining, or perhaps specify dates for your trip? Is there any feedback on this plan or any other preferences you'd like to save?
--------------------------------------------------


"Great! For your trip to Sunnyvale, I recommend the following:\n\n*   **Activity:** Computer History Museum\n*   **Restaurant:** Merit Vegan Restaurant (keeping in mind your vegetarian preference!)\n\nWould you like to explore more options for activities or dining, or perhaps specify dates for your trip? Is there any feedback on this plan or any other preferences you'd like to save?"

---
## 6. The REAL Test: Kernel Restart & New Session 🔄

This is the ultimate test. We will prove **both** session isolation and user-level memory.

**First, do the following:**
1.  Go to the menu at the top of Colab.
2.  Click **Runtime > Restart session...** and confirm.
3.  Wait for the kernel to reconnect.
4.  **Manually re-run all the code cells** from "Part 0" down to here to re-initialize everything.
5.  Once all cells above are run, run the *next* code cell.

We will use a **NEW session_id** (`session_b`). The agent should find *no conversation history*, but it should *immediately* find your user preference from the `user_preferences.db`!

In [ ]:
# Turn 4: (AFTER KERNEL RESTART)
# We use the SAME user_id but a NEW session_id
MY_NEW_SESSION_ID = f"{MY_USER_ID}_session_b"

print(f"--- Starting a new conversation in {MY_NEW_SESSION_ID} ---")

query4 = "Hi, I'm back. Can you plan a trip for me?"
await run_persistent_query_async(query4, MY_USER_ID, MY_NEW_SESSION_ID)

--- Starting a new conversation in colab_user_001_session_b ---

--- 🗣️ You: Hi, I'm back. Can you plan a trip for me? ---
🤖 Agent: 
[SYSTEM] No existing conversation. Creating new one: colab_user_001_session_b



[DEBUG] Tool 'recall_user_preferences' connecting to user_preferences.db for user colab_user_001


Great to have you back!

Based on your previous preference for vegetarian options, I've planned the following for you:

*   **Activity:** Computer History Museum
*   **Restaurant:** Merit Vegan Cuisine

How does this sound? Is there anything you'd like to change, or any new preferences I should remember for next time?
--------------------------------------------------


"Great to have you back!\n\nBased on your previous preference for vegetarian options, I've planned the following for you:\n\n*   **Activity:** Computer History Museum\n*   **Restaurant:** Merit Vegan Cuisine\n\nHow does this sound? Is there anything you'd like to change, or any new preferences I should remember for next time?"

---
## 🎉 Congratulations! 🎉

If everything worked, the agent in the last step did two things:
1.  It created a **new conversation** (e.g., `session_b`) because `session_a`'s in-memory context was gone.
2.  It *immediately* called `recall_user_preferences` and **found your vegetarian preference** from the `user_preferences.db`, even though it had never spoken to you in this new session!

You have successfully implemented the most powerful and scalable memory pattern:

* **Conversational Memory** using `DatabaseSessionService`.
* **Persistent User-Level Memory** using a separate, external database.